In [1]:
EXPT_NAME = "Exp01: Shell and Tube HE"
import math

import platform
from datetime import datetime
print("**************************************")
print(EXPT_NAME)
print("Roll No: 120CH0057, Name: Siddharth Gautam")
print("Running On: Python ", platform.python_version(), " on: ", str(datetime.now()))
print("**************************************")
print("")

shell_pass = 1
tube_pass = 2
MFR_Butanol = 100000
C_P_Butanol = 2369
C_P_Water = 4186
rho_Butanol = 776
rho_Water = 996
mu_Butanol = 0.95 * 10 ** (-3)
mu_Water = 0.89 * 10 ** (-3)
k_Butanol = 0.152 
k_Water = 0.601

T_hi = 95
T_ho = 40
t_ci = 25
t_co = 35
# U = 100 Btu/lb ft^2 F = 567 W/m^2 K
U_assume = 567
# 6 m tubes
tube_length = 6 
# 1 in OD 
outer_diameter = 0.0254
pitch = 0.0381

MFR_Water = MFR_Butanol * (T_hi - T_ho) * C_P_Butanol / (C_P_Water * (t_co - t_ci))
print("MFR of Water: " + str(MFR_Water/1000) + " T/hr")

# Determine LMTD correction factor for given N-2M STHE
# Formula from: https://cheguide.com/lmtd_charts.html
def LMTD_correction(N, M, R, S):
    K1 = math.sqrt(S**2 + 1) / (S - 1)
    K2 = abs(((1 - R*S)/(1 - R))**(1/N))
    return 1/(K1*math.log(K2) / math.log(abs((1 + K2 - K1 + K2*K1) / (1 + K2 + K1 - K1*K2))))

R = (T_hi - T_ho) / (t_co - t_ci)
S = (t_co - t_ci) / (T_hi - t_ci)

LMTD = ((T_hi - t_co) - (T_ho - t_ci)) / math.log((T_hi - t_co) / (T_ho - t_ci))
F = LMTD_correction(shell_pass, tube_pass / 2, R, S)
LMTD_corr = LMTD * F

print("LMTD: " + str(LMTD))
print("LMTD correction factor for |R = " + str(R) + "|S = " + str(round(S, 2)) + "| : " + str(F))
print("LMTD (corrected) " + str(LMTD_corr))

Q = (MFR_Butanol/3600) * C_P_Butanol * (T_hi - T_ho)
print("Q: " + str(Q) + " J/s")

A = Q / (U_assume * LMTD_corr)
print("A: " + str(A) + " m^2")

N_tubes = A / (math.pi * outer_diameter * tube_length)
print("Number of tubes: " + str(N_tubes))

N_actual_tubes = 460
A_new = N_actual_tubes * math.pi * outer_diameter * tube_length
print("A_new: " + str(A_new) + " m^2")
U_corr = Q / (A_new * LMTD_corr)
U_err = abs((U_corr - U_assume) / U_assume)*100
print("Corrected U: " + str(U_corr) + " W/m^2 K")
print("Error in U: " + str(round(U_err, 2)) + "%")

# Tube side HTC
A_1t = (N_actual_tubes / 2) * ((math.pi * (outer_diameter * 0.8) ** 2) / 4)
print("A_1t: " + str(A_1t) + " m^2")

v_tubes = (MFR_Butanol / 3600) / (rho_Butanol * A_1t)
print("v_tube: " + str(v_tubes) + " m/s")

N_Re = (0.8 * outer_diameter) * v_tubes * rho_Butanol / mu_Butanol
print("Reynolds No. (Tube Side): " + str(N_Re))
N_Pr = C_P_Butanol * mu_Butanol / k_Butanol
print("Prandtl No. (Tube Side): " + str(N_Pr))

h_i = (k_Butanol / (0.8 * outer_diameter)) * 0.023 * (N_Re ** (0.8)) * (N_Pr ** (0.4))
print("Tube Side HTC: " + str(h_i) + " W / m^2 K")

# Shell Side HTC
# 33 in shell diameter
shell_diameter = 0.8382
baffle_spacing = shell_diameter / 5
n_baffles = math.floor(shell_diameter / baffle_spacing)
baffle_length = 0.75 * shell_diameter
D_e = 4 * (pitch * pitch * math.sin(math.pi/3) - (math.pi/4)*(outer_diameter**2)) / (math.pi * outer_diameter)
print("Equivalent Diameter of Shell: " + str(D_e) + " m")
A_shell = (pitch - outer_diameter)*shell_diameter*baffle_length / pitch
print("Shell flow area: " + str(A_shell) + " m^2")
v_shell = (MFR_Water / 3600) / (rho_Water * A_shell)
print("v_shell: " + str(v_shell) + " m/s")

N_Re = (D_e) * v_shell * rho_Water / mu_Water
print("Reynolds No. (Shell Side): " + str(N_Re))
N_Pr = C_P_Water * mu_Water / k_Water
print("Prandtl No. (Shell Side): " + str(N_Pr))
h_s = (k_Butanol / D_e) * 0.36 * (N_Re ** (0.55)) * (N_Pr ** (0.33))
print("Shell Side HTC: " + str(h_s) + " W / m^2 K")

U_actual = 1/((1/h_s) + (1/h_i))
print("Actual U: " + str(U_actual))

# Dirt Factor
R_d = (1/U_actual) - (1/U_corr)
print("Dirt Factor: " + str(R_d))

# Compile Results
results = { "MFR Water (T/hr)": MFR_Water/1000, "LMTD (C)": LMTD_corr, "Corrected U (W/m^2K)" : U_corr, "Actual U (W/m^2 K)" : U_actual, "Dirt Factor": R_d, "Tube Diameter (OD, m)": outer_diameter, "Tube Length (m)": tube_length, "Number of Tubes": N_actual_tubes, "Shell Diameter (m)": shell_diameter, "Number of Baffles": n_baffles, "Baffle Spacing (m)": baffle_spacing, "Baffle Length (m)": baffle_length }
import pandas as pd

df = pd.DataFrame(results.items(), columns=['Parameter', 'Value'])
display(df)

**************************************
Exp01: Shell and Tube HE
Roll No: 120CH0057, Name: Siddharth Gautam
Running On: Python  3.11.1  on:  2023-04-25 15:36:46.473407
**************************************

MFR of Water: 311.2637362637363 T/hr
LMTD: 32.46063842000168
LMTD correction factor for |R = 5.5|S = 0.14| : 0.9386931805459822
LMTD (corrected) 30.47057992102448
Q: 3619305.555555556 J/s
A: 209.48911916393698 m^2
Number of tubes: 437.54893489380146
A_new: 220.23821138725884 m^2
Corrected U: 539.3266219234467 W/m^2 K
Error in U: 4.88%
A_1t: 0.07458734092315168 m^2
v_tube: 0.47992199400988766 m/s
Reynolds No. (Tube Side): 7965.856396406308
Prandtl No. (Tube Side): 14.80625
Tube Side HTC: 668.054248029875 W / m^2 K
Equivalent Diameter of Shell: 0.03761689274671084 m
Shell flow area: 0.17564481 m^2
v_shell: 0.4942325737275225 m/s
Reynolds No. (Shell Side): 20805.761508952484
Prandtl No. (Shell Side): 6.198901830282862
Shell Side HTC: 629.8367957691195 W / m^2 K
Actual U: 324.1914249962

,Parameter,Value
0,MFR Water (T/hr),311.263736
1,LMTD (C),30.470580
2,Corrected U (W/m^2K),539.326622
3,Actual U (W/m^2 K),324.191425
4,Dirt Factor,0.001230
5,"Tube Diameter (OD, m)",0.025400
6,Tube Length (m),6.000000
7,Number of Tubes,460.000000
8,Shell Diameter (m),0.838200
9,Number of Baffles,5.000000
